<p style="text-align:center; font-size:48px;">
LA ADICCIÓN A REDES SOCIALES
</h1>
</p>


<p align="center">
  <img src="src/img/redes_sociales_sin_cabezas.jpg" alt="Adicción a redes sociales" width="600" />
</p>

**📦Librerias necesarias**

In [7]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV




ModuleNotFoundError: No module named 'imblearn'

**1. Visualizacion general de mis datos**

Nos encontramos con un Dataset aparentemente limpio, detectamos una columna ID, hacemos una copia de seguridad y quitamos esta columna que no nos sera util. Nos encontramos con un problema supervisado de clasificacion multiclases

In [ ]:
student = pd.read_csv("./data/Students_Social_Media_Addiction.csv", sep = ",")
student.head()

,Student_ID,Age,Gender,Academic_Level,Country,Avg_Daily_Usage_Hours,Most_Used_Platform,Affects_Academic_Performance,Sleep_Hours_Per_Night,Mental_Health_Score,Relationship_Status,Conflicts_Over_Social_Media,Addicted_Score
0,1,19,Female,Undergraduate,Bangladesh,5.2,Instagram,Yes,6.5,6,In Relationship,3,8
1,2,22,Male,Graduate,India,2.1,Twitter,No,7.5,8,Single,0,3
2,3,20,Female,Undergraduate,USA,6.0,TikTok,Yes,5.0,5,Complicated,4,9
3,4,18,Male,High School,UK,3.0,YouTube,No,7.0,7,Single,1,4
4,5,21,Male,Graduate,Canada,4.5,Facebook,Yes,6.0,6,In Relationship,2,7


In [ ]:
student.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Student_ID                    705 non-null    int64  
 1   Age                           705 non-null    int64  
 2   Gender                        705 non-null    object 
 3   Academic_Level                705 non-null    object 
 4   Country                       705 non-null    object 
 5   Avg_Daily_Usage_Hours         705 non-null    float64
 6   Most_Used_Platform            705 non-null    object 
 7   Affects_Academic_Performance  705 non-null    object 
 8   Sleep_Hours_Per_Night         705 non-null    float64
 9   Mental_Health_Score           705 non-null    int64  
 10  Relationship_Status           705 non-null    object 
 11  Conflicts_Over_Social_Media   705 non-null    int64  
 12  Addicted_Score                705 non-null    int64  
dtypes: fl

In [ ]:
to_drop= ["Student_ID","Gender","Academic_Level","Avg_Daily_Usage_Hours", "Most_Used_Platform","Affects_Academic_Performance","Mental_Health_Score", "Relationship_Status", "Conflicts_Over_Social_Media"]

In [ ]:
df= student.copy()

In [ ]:
df.drop(to_drop, axis=1,inplace=True)

In [ ]:
df

,Age,Country,Sleep_Hours_Per_Night,Addicted_Score
0,19,Bangladesh,6.5,8
1,22,India,7.5,3
2,20,USA,5.0,9
3,18,UK,7.0,4
4,21,Canada,6.0,7
...,...,...,...,...
700,20,Italy,7.2,5
701,23,Russia,5.9,9
702,21,China,6.7,7
703,24,Japan,7.5,4


**2. Separamos nuestro dataset, e identificamos el target:**

In [ ]:
train_set, test_set = train_test_split(df, test_size= 0.2, random_state= 42)
target = "Addicted_Score"

**3. Miremos el target:**

Nos encontramos frente a un problema de clasificacion multiclase, no balanceado. 

In [ ]:
df[target].value_counts(normalize=True)

Addicted_Score
7    0.296454
8    0.204255
5    0.192908
4    0.117730
6    0.086525
9    0.078014
3    0.022695
2    0.001418
Name: proportion, dtype: float64

In [ ]:
# TRANSFORMACIONES - importante, tenemos en cuenta posibles outliers a futuro 
# COUNTRY 

country_to_continent = {
    #Asia
    'Afghanistan': 'Asia', 'Armenia': 'Asia', 'Azerbaijan': 'Asia', 'Bahrain': 'Asia',
    'Bangladesh': 'Asia', 'Bhutan': 'Asia', 'Cyprus': 'Asia', 'Georgia': 'Asia',
    'India': 'Asia', 'Indonesia': 'Asia', 'Iraq': 'Asia', 'Israel': 'Asia',
    'Japan': 'Asia', 'Jordan': 'Asia', 'Kazakhstan': 'Asia', 'Kuwait': 'Asia',
    'Kyrgyzstan': 'Asia', 'Lebanon': 'Asia', 'Malaysia': 'Asia', 'Maldives': 'Asia',
    'Nepal': 'Asia', 'Oman': 'Asia', 'Pakistan': 'Asia', 'Philippines': 'Asia',
    'Qatar': 'Asia', 'Singapore': 'Asia', 'South Korea': 'Asia', 'Sri Lanka': 'Asia',
    'Syria': 'Asia', 'Taiwan': 'Asia', 'Tajikistan': 'Asia', 'Thailand': 'Asia',
    'Turkey': 'Asia', 'UAE': 'Asia', 'Uzbekistan': 'Asia', 'Vietnam': 'Asia',
    'Yemen': 'Asia',
    # Africa
    'Egypt': 'Africa', 'Ghana': 'Africa', 'Kenya': 'Africa',
    'Morocco': 'Africa', 'Nigeria': 'Africa', 'South Africa': 'Africa', 'Argelia': 'Africa', 'Angola': 'Africa','Benin': 'Africa', 'Botswana': 'Africa','Burkina Faso': 'Africa','Burundi': 'Africa', 'Cape Verde': 'Africa', 'Cameroon': 'Africa', 'Chad': 'Africa',
    'Comoros': 'Africa','Republic of the Congo': 'Africa',
    # North America
    'Bahamas':'North America', 'Canada': 'North America', 'Costa Rica': 'North America','Jamaica': 'North America', 'Mexico': 'North America', 'Panama': 'North America',
    'Trinidad': 'North America', 'USA': 'North America',
    # South America
    'Argentina': 'South America', 'Bolivia': 'South America', 'Brazil': 'South America',
    'Chile': 'South America', 'Colombia': 'South America', 'Ecuador': 'South America',
    'Paraguay': 'South America', 'Peru': 'South America', 'Uruguay': 'South America',
    'Venezuela': 'South America',
    # Europe
    'Albania': 'Europe', 'Andorra': 'Europe', 'Austria': 'Europe', 'Belarus': 'Europe',
    'Belgium': 'Europe', 'Bosnia': 'Europe', 'Bulgaria': 'Europe', 'Croatia': 'Europe',
    'Czech Republic': 'Europe', 'Denmark': 'Europe', 'Estonia': 'Europe', 'Finland': 'Europe',
    'France': 'Europe', 'Germany': 'Europe', 'Greece': 'Europe', 'Hungary': 'Europe',
    'Iceland': 'Europe', 'Ireland': 'Europe', 'Italy': 'Europe', 'Kosovo': 'Europe',
    'Latvia': 'Europe', 'Liechtenstein': 'Europe', 'Lithuania': 'Europe', 'Luxembourg': 'Europe',
    'Malta': 'Europe', 'Moldova': 'Europe', 'Monaco': 'Europe', 'Montenegro': 'Europe',
    'Netherlands': 'Europe', 'North Macedonia': 'Europe', 'Norway': 'Europe', 'Poland': 'Europe',
    'Portugal': 'Europe', 'Romania': 'Europe', 'San Marino': 'Europe', 'Serbia': 'Europe',
    'Slovakia': 'Europe', 'Slovenia': 'Europe', 'Spain': 'Europe', 'Sweden': 'Europe',
    'Switzerland': 'Europe', 'Ukraine': 'Europe', 'UK': 'Europe', 'Vatican City': 'Europe',
    # Oceania
    'Australia': 'Oceania', 'New Zealand': 'Oceania'
}

train_set['Continent'] = train_set['Country'].map(country_to_continent)
test_set['Continent'] = test_set['Country'].map(country_to_continent)
train_set.drop(columns="Country", inplace=True)
test_set.drop(columns="Country", inplace=True)


In [ ]:
transformed_cat = ["Continent", "Age" ]
for col in transformed_cat:
    train_set= pd.get_dummies(train_set, columns=[col], drop_first=True, dtype=int)
    test_set= pd.get_dummies(test_set, columns=[col], drop_first=True, dtype=int)

In [ ]:
#verificamos que tenemos todas las mismas columnas en test_set que en train.
train_set.columns

Index(['Sleep_Hours_Per_Night', 'Addicted_Score', 'Continent_Asia',
       'Continent_Europe', 'Continent_North America', 'Continent_Oceania',
       'Continent_South America', 'Age_19', 'Age_20', 'Age_21', 'Age_22',
       'Age_23', 'Age_24'],
      dtype='object')

In [ ]:
train_set

,Sleep_Hours_Per_Night,Addicted_Score,Continent_Asia,Continent_Europe,Continent_North America,Continent_Oceania,Continent_South America,Age_19,Age_20,Age_21,Age_22,Age_23,Age_24
24,6.9,5,0,1,0,0,0,0,0,0,1,0,0
155,7.4,8,1,0,0,0,0,0,1,0,0,0,0
210,4.4,8,1,0,0,0,0,0,1,0,0,0,0
260,5.6,9,0,0,1,0,0,1,0,0,0,0,0
424,8.9,4,0,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,6.8,5,0,1,0,0,0,0,0,1,0,0,0
106,6.6,6,1,0,0,0,0,0,0,1,0,0,0
270,7.3,5,0,0,0,1,0,0,1,0,0,0,0
435,7.8,7,1,0,0,0,0,0,0,0,1,0,0


In [ ]:
len(train_set.columns.to_list())

13

In [ ]:
len(test_set.columns.to_list())

13

In [ ]:

X_train = train_set.drop(target, axis = 1)
y_train = train_set[target]
X_test = test_set.drop(target, axis = 1)
y_test = test_set[target]

In [ ]:
#Aplicamos un SMOTE para el balanceo del target, aseguremos asi el buen entrenamiento tambien de las clases minoritarias
sm = SMOTE(random_state=42)
X_smoted, y_smoted = sm.fit_resample(X_train, y_train)

rfc_smote = RandomForestClassifier(random_state=42, )
baseline_smoted = np.mean(cross_val_score(rfc_smote,X_smoted,y_smoted, scoring = "balanced_accuracy", cv = 5))
print(f"Baseline_smoted: {baseline_smoted}%")

Baseline_smoted: 0.6904761904761905%


In [ ]:

params = {
    'max_depth': [ 5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'n_estimators' : [50, 100, 200]
}

dt = RandomForestClassifier(random_state=42, class_weight='balanced')

grid_search = GridSearchCV(
        dt, 
        param_grid=params, 
        scoring='balanced_accuracy', 
        cv=5, 
        n_jobs=-1
        )
grid_search.fit(X_smoted, y_smoted)


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [5, 10, 15, None],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='balanced_accuracy')

In [ ]:
grid_search.best_score_

np.float64(0.7381716322892794)

In [ ]:
grid_search.best_estimator_

RandomForestClassifier(class_weight='balanced', max_depth=15,
                       min_samples_leaf=2, n_estimators=50, random_state=42)

In [ ]:
grid_search.best_params_

{'max_depth': 15,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 50}

In [ ]:
best_model = grid_search.best_estimator_  # el modelo entrenado
y_pred = best_model.predict(X_test)
y_pred

array([4, 7, 8, 3, 4, 9, 3, 6, 5, 8, 5, 8, 9, 7, 4, 8, 6, 7, 6, 6, 7, 8,
       8, 9, 7, 5, 6, 8, 5, 8, 9, 6, 4, 6, 8, 8, 7, 5, 7, 5, 8, 5, 6, 6,
       9, 5, 8, 6, 4, 8, 7, 6, 4, 5, 7, 8, 5, 7, 8, 7, 6, 5, 5, 7, 5, 5,
       8, 6, 8, 8, 8, 8, 5, 4, 5, 5, 7, 8, 9, 7, 7, 7, 8, 9, 7, 5, 8, 7,
       8, 7, 5, 9, 8, 7, 4, 8, 4, 5, 7, 4, 4, 3, 7, 7, 4, 5, 9, 4, 8, 7,
       7, 3, 6, 6, 8, 6, 8, 6, 5, 8, 8, 6, 9, 7, 8, 9, 7, 4, 4, 7, 7, 8,
       4, 8, 4, 9, 4, 7, 9, 5, 7])

In [ ]:
# Guardar el modelo
import joblib
joblib.dump(best_model, '.model.pkl')

['.model.pkl']